# Importation et manipulation de données spatiales {#sec-chap01}

Dans le chapitre, nous abordons quelques formats d'images ainsi que leur lecture.

::: bloc_package
::: bloc_package-header
::: bloc_package-icon
:::

**Liste des *librairies* utilisées dans ce chapitre**
:::

::: bloc_package-body
-   Pour importer et manipuler des fichiers géographiques :
    -   `sf` pour importer et manipuler des données vectorielles.
    -   `Rasterio` pour importer des images.
    -   `NumPy` pour manipuler des données matricielles.
-   Pour construire des cartes et des graphiques :
    -   `tmap` est certainement le meilleur *package* pour la cartographie.
    -   `Matplotlib` pour construire des graphiques.
:::
:::

## Bases de Quarto... {#sec-010}

Voici comment faire une liste :

-   **texte en gras**
-   *texte en italique*
    -   ***Gras et italique***
-   R^2^ et NO~2~
-   [Petites majuscules]{.smallcaps}
-   Pour un lien Web, [Département de géomatique appliquée](https://www.usherbrooke.ca/geomatique/).

Voici comment intégrer des équations LaTeX. La formule de la distance euclidienne (@eq-DistEuc).
Pour écrire directement une équation dans le texte, il suffit de $E = mc^2$.

$$
 d_{ij} = 2R \cdot \text{ arcsin} \left( \sqrt{\text{sin}^2 \left( \frac{\delta _i - \delta _j}{2} \right) + \text{cos }\delta _i \cdot \text{cos }\delta _j \cdot \text{sin}^2 \left( \frac{\phi _i - \phi _j}{2} \right)} \right)
$$ {#eq-DistLongLat}

$$
 d_{ij} = \sqrt{(x_i-x_j)^2+(y_i-y_j)^2}
$$ {#eq-DistEuc}

-   Intégrer une figure (image) et l'appeler dans le texte (@fig-downloaffromgit). À la @fig-downloaffromgit, ... Notez que la numérotation des figures, des tableaux, des équations est automatique.

![Téléchargement de l'intégralité du livre](images/introduction/download_github.png){#fig-downloaffromgit width="40%" fig-align="center"}


-   Les références sont au format BibTeX. Par exemple, vous pouvez les télécharger de Google Scholar et les intégrer à la fin du fichier references.bib. Voici comment intégrer des références [@mather2022computer; @richards2022remote]. Selon Ferdinand Boon et Guy Rochon [-@PrecisTeleVol1].


Le code R ci-dessous permet de faire un tableau dans un chunk! Pour appeler le tableau (@tbl-TabMatricesSpatiales).


## Importation d'images {#sec-011}

La première étape avant tout traitement est d'accéder à la donnée image pour qu'elle soit manipulée par le programme Python. L'imagerie satellite présente certains défis notamment en raison de la taille parfois très importante des images. Il existe maintenant certaines librairies, comme 🔖[Xarray](https://docs.xarray.dev/en/stable/), qui on cherchées à optimiser la lecture et l'écriture de grandes images. Il est donc conseiller de toujours garder un oeil sur l'espace mémoire occupé par les variables Python reliées aux images. La librairie principale en géomatique qui va nous permettre d'importer (et d'exporter) de l'imagerie est la librairie [GDAL](https://gdal.org) qui rassemble la plupart des formats sous forme de *driver*. 

Dans le domaine de la géomatique, il faut prêter attention à trois caractéristiques principales des images:
1. **La matrice des données** elle-même qui contient les valeurs brutes des pixels. Cette matrice sera souvent un cube à trois dimensions. En Python, ce cube sera le plus souvent un objet de la librairie [NumPy](https://numpy.org/) (voir section).
2. **La dynamique des images** c.à.d le format de stockage des valeurs individuelles (octet, entier, double, etc.). Ce format décide principalement de la résolution radiométrique et des valeurs minimales et maximales supportées. 
3. **La métadonnée** qui va transporter sur l'information auxiliaire de l'image comme les dimensions et la position de l'image, la date, etc. Cette donnée auxiliaire prendra souvent la forme d'un dictionnaire Python. 

Les différents formats se distinguent principalement sur comment ces trois caractéristiques sont gérées.

### Formats de données {#sec-0111}

Il existe de nombreux formats pour la donnée de type image parfois appelé donnée matricielle ou donnée *raster*. La librairie GDAL rassemble la plupart des formats matriciels rencontrés en géomatique (voir 🔖[Raster drivers — GDAL documentation](https://gdal.org/en/latest/drivers/raster/index.html) pour une liste complète).

On peut distinguer deux grandes familles de format:
1. Les formats de type **RVB** issus de l'imagerie numérique grand publique comme 🔖[JPEG](https://gdal.org/en/latest/drivers/raster/jpeg.html#raster-jpeg), [png](https://gdal.org/en/latest/drivers/raster/png.html#raster-png), etc. Ces formats ne supportent généralement que trois bandes au maximum (rouge, vert et bleu) et des valeurs de niveaux de gris entre 0 et 255 (format dit 8 bit).
2. **Les géo-formats** issus des domaines scientifiques ou techniques comme GeoTIFF, HDF5, etc.

Les formats RVB restent très utilisés en Python notamment dans les librairies dites de vision par ordinateur (*Computer Vision*) comme opencv et sickit-image ainsi que les grandes librairies en apprentissage profond (PyTorch, Tensorflow).  


#### Formats de type RVB {#sec-01111}

Les premiers formats pour de l'imagerie à une bande (monochrome) et à trois bandes (image couleur rouge-vert-bleu) sont issus du domaine des sciences de l'ordinateur. On trouvera, entre autres, les formats pbm, png et jpeg. Ces formats supportent peu de métadonnées dans un entête (*header*) très limité. Ces formats restent très populaires dans le domaine de la vision par ordinateur et sont très utilisés en apprentissage profond particulier. Pour la lecture des images RVB, on peut utiliser Rasterio, [PIL](https://he-arc.github.io/livre-python/pillow/index.html) ou [OpenCV](https://docs.opencv.org/4.10.0/index.html).

##### Lecture avec la librairie OpenCV
La librairie PIL retourne un objet de type `PngImageFile`, l'affichage de l'image se fait directement dans la cellule de sortie.

In [ ]:
#| label: tbl-lecture-PIL-PNG
#| tbl-cap: Lecture d'une image en format PNG avec PIL
#| echo: false
#| eval: false
#| warning: false
!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG
from PIL import Image
img = Image.open('/content/modis-aqua.PNG')
img

##### Lecture avec la librairie OpenCV
La librairie [OpenCV](https://docs.opencv.org/4.10.0/index.html) est aussi très populaire en vision par ordinateur. La fonction `imread` donne directement un objet de type NumPy en sortie.

In [ ]:
#| label: tbl-lecture-opencv-PNG
#| tbl-cap: Lecture d'une image en format PNG avec OpenCV
#| echo: false
#| eval: false
#| warning: false
!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG
import cv2
img = cv2.imread('/content/modis-aqua.PNG')
img

##### Lecture avec la librairie RasterIO

Rien ne nous empêche de lire une image de format RVB avec [RasterIO](https://rasterio.readthedocs.io/en/stable/) comme décrit dans (@tbl-lecture-rasterio-PNG). Vous noterez cependant les avertissements concernant l'absence de géoréférence pour ce type d'image.

In [ ]:
#| label: tbl-lecture-rasterio-PNG
#| tbl-cap: Lecture d'une image en format PNG avec OpenCV
#| echo: false
#| eval: false
#| warning: false
!wget https://raw.githubusercontent.com/sfoucher/TraitementImagesPythonVol1/refs/heads/main/images/modis-aqua.PNG
import rasterio
img= rasterio.open('/content/modis-aqua.PNG')
img

#### Le format GeoTiff {#sec-01112}

Le format GeoTIFF est une extension du format TIFF (Tagged Image File Format) qui permet d'incorporer des métadonnées géospatiales directement dans un fichier image. Développé initialement par Dr. Niles Ritter au Jet Propulsion Laboratory de la [NASA](https://www.earthdata.nasa.gov/esdis/esco/standards-and-practices/geotiff){target="_blank"} dans les années 1990, GeoTIFF est devenu un standard de facto pour le stockage et l'échange d'images géoréférencées dans les domaines de la télédétection et des systèmes d'information géographique (SIG). Ce format supporte plus que trois bandes aussi longtemps que ces bandes sont de même dimension.

Le format GeoTIFF est très utilisé et est largement supporté par les bibliothèques et logiciels géospatiaux, notamment GDAL (Geospatial Data Abstraction Library), qui offre des capacités de lecture et d'écriture pour ce format. Cette compatibilité étendue a contribué à son adoption généralisée dans la communauté géospatiale.

##### Standardisation par l'OGC {#sec-011121}

Le standard GeoTIFF proposé par l'Open Geospatial Consortium (OGC) en 2019 formalise et étend les spécifications originales du format GeoTIFF, offrant une norme robuste pour l'échange d'images géoréférencées. Cette standardisation, connue sous le nom d'OGC GeoTIFF 1.1 [-@OGCGeoTIFF], apporte plusieurs améliorations et clarifications importantes.


#### Le format COG

Une innovation récente dans l'écosystème GeoTIFF est le format *Cloud Optimized GeoTIFF* ([COG](http://cogeo.org/)), conçu pour faciliter l'utilisation de fichiers GeoTIFF hébergés sur des serveurs web HTTP. Le COG permet aux utilisateurs et aux logiciels d'accéder à des parties spécifiques du fichier sans avoir à le télécharger entièrement, ce qui est particulièrement utile pour les applications basées sur le cloud.

### Métadonnées des images {#sec-0112}

### Données en géoscience {#sec-0113}

Calibration, unités, données manquantes, données éparses.

netcdf, xarray, GRIB.

Données météos, exemple avec SWOT.

## Importation de données vectorielles {#sec-012}

### Importation d'un fichier *shapefile* {#sec-0121}

### Importation d'une couche dans un *GeoPackage* {#sec-0122}

### Importation d'une couche dans une *geodatabase* d'ESRI {#sec-0123}

### Importation d'un fichier *shapefile* {#sec-0124}


## Manipulation d'images {#sec-013}

### Manipulation d'une matrice {#sec-0131}

- Numpy

### Mosaïquage, masquage et découpage {#sec-0132}

### Changement de projection cartographique {#sec-0133}

### Recalage d'images et co-registration {#sec-0134}

### Requêtes attributaires {#sec-0135}

## Manipulation de données vectorielles {#sec-014}

## Quiz de révision du chapitre {#sec-015}

## Exercices de révision {#sec-016}

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 1.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08011].
:::
:::

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 2.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08012].
:::
:::

::: bloc_exercice
::: bloc_exercice-header
::: bloc_exercice-icon
:::

**Exercice 3.** À compléter
:::

::: bloc_exercice-body

In [ ]:
#| echo: true
#| warning: false
#| eval: false

# ...
# à compléter

Correction à la [section @sec-08013].
:::
:::